In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import type_score,get_year_ipcc,get_year_ipbes,get_xy,check_doi_glutton,normalize,aplatir,remove_too_long
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache

In [2]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_visualization.json')
ipcc_chap=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\ipcc_chap.jsonl', lines=True)

In [3]:
df_ipcc

,year,url,doi,title,author,freq,countries,concepts,sdg,topics,...,title_OA,year_OA,authors_name,rors,institutions_names,locations_names,locations_ids,type,type_crossref,test_glutton
0,2019,https://www.frontiersin.org/research-topics/82...,10.3389/978-2-88963-118-6,{OceanObs'19: An Ocean of Opportunity. Volume 1},None,1,[None],"[Volume (thermodynamics), Oceanography, Enviro...","[{'id': '14', 'name': 'Life below water'}]",[Marine Biology and Ecology Research],...,Oceanobs'19: An Ocean of Opportunity. Volume I,2020,[None],[None],[None],Frontiers research topics,1664-8714,book,book,True
1,2000,None,10.1007/978-94-015-9612-1,"{Eduard Br{\""{u}}ckner – The Sources and Conse...",None,1,"[CA, UA]","[Geographer, Climate change, Politics, Climato...","[{'id': '13', 'name': 'Climate action'}]","[Geology and Paleoclimatology Research, Climat...",...,Eduard Brückner — The Sources and Consequences...,2000,"[[Nico Stehr, [CA]], [Hans von Storch, [UA]]]","[[https://ror.org/03rmrcq20, CA], [https://ror...","[[University of British Columbia, CA], [Instit...",None,None,book,book,True
2,1900,None,10.1002/andp.19003081208,"{{\""{U}}ber die Bedeutung des Wasserdampfes un...","{\AA}ngstr{\""{o}}m, Knut",1,[US],"[Physics, Humanities, Angstrom, Crystallograph...",None,"[Atmospheric Ozone and Climate, Atmospheric ch...",...,Ueber die Bedeutung des Wasserdampfes und der ...,1900,"[[Knut Ångström, [US]]]","[[https://ror.org/03wbhj477, US]]","[[Upsala College, US]]",Annalen der Physik,0003-3804,article,journal-article,True
3,1964,https://www.tandfonline.com/doi/full/10.3402/t...,10.3402/tellusa.v16i1.8885,{The parameters of atmospheric turbidity},"{\AA}ngstr{\""{o}}m, Anders",1,[US],"[Turbidity, Environmental science, Meteorology...",None,[Ocean Waves and Remote Sensing],...,The parameters of atmospheric turbidity,1964,"[[Anders Ångström, [US]]]","[[https://ror.org/03a8x4b06, US]]","[[Eppley Foundation for Research, US]]",Tellus A Dynamic Meteorology and Oceanography,0280-6495,article,journal-article,True
4,1929,https://www.tandfonline.com/doi/full/10.1080/2...,10.1080/20014422.1929.11880498,{On the Atmospheric Transmission of Sun Radiat...,"{\AA}ngstr{\""{o}}m, Anders",1,[None],"[Atmospheric dust, Radiation, Transmission (te...","[{'id': '14', 'name': 'Life below water'}]",[Atmospheric aerosols and clouds],...,On the Atmospheric Transmission of Sun Radiati...,1929,"[[Anders Ångström, []]]",[],[],Geografiska Annaler,1651-3215,article,journal-article,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72829,2018,https://doi.org/10.1016%2Fj.apenergy.2017.07.036,10.1016/j.apenergy.2017.07.036,The water-food-energy nexus optimization appro...,Jie Zhang and Pietro Elia Campana and Tian Yao...,1,"[SE, US]","[Nexus (standard), Agriculture, Yield (enginee...",None,"[Water-Energy-Food Nexus Studies, Water resour...",...,The water-food-energy nexus optimization appro...,2017,"[[Jie Zhang, [US]], [Pietro Elia Campana, [SE]...","[[https://ror.org/047s2c258, US], [https://ror...","[[University of Maryland, College Park, US], [...",Applied Energy,0306-2619,article,journal-article,True
72830,2019,https://doi.org/10.1029%2F2019rg000644,10.1029/2019rg000644,A Review of the Role of the Atlantic Meridiona...,Rong Zhang and Rowan Sutton and Gokhan Danabas...,1,"[GB, US]","[Atlantic multidecadal oscillation, Climatolog...","[{'id': '13', 'name': 'Climate action'}]","[Climate variability and models, Tropical and ...",...,A Review of the Role of the Atlantic Meridiona...,2019,"[[Rong Zhang, [US]], [Rowan Sutton, [GB]], [Gö...","[[https://ror.org/03vmn1898, US], [https://ror...","[[NOAA Geophysical Fluid Dynamics Laboratory, ...",Reviews of Geophysics,1944-9208,review,journal-article,True
72831,2013,https://doi.org/10.1016%2Fj.enpol.2013.08.092,10.1016/j.enpol.2013.08.092,To what extent does wind power deployment affe...,Xiaoli Zhao and Sufang Zhang and Yasheng Zou a...,1,[CN],"[Software deployment, Revenue, Renewable energ...","[{'id': 

In [4]:
ipcc_chap['ipcc']=ipcc_chap.apply(lambda row: {'name': str(row.chap), 'wg': str(row.wg), 'chap': str(row.chap)[-2:]},axis=1)

In [5]:
ipcc_chap=ipcc_chap.groupby(by='doi').agg({'ipcc': lambda x: list(x)}).reset_index()

In [6]:
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipcc.loc[:,'rors']=df_ipcc.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [7]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0], 'freq':'sum'})
df_ipcc_group=df_ipcc_group.reset_index()

In [8]:
df_ipcc_group=pd.merge(df_ipcc_group,ipcc_chap, on='doi', how='left')

In [9]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [10]:
len(df_ipcc_group)

52471

In [11]:
len(df_ipcc_group.doi.drop_duplicates())==len(df_ipcc_group.doi)

True

In [13]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.jsonl', lines=True)

In [14]:
df_ipbes

,doi,title,date,creators,tags,countries,concepts,sdg,topics,is_OA_available,year_OA,authors_name,rors,institutions_names,locations_names,locations_ids,type,type_crossref,year
0,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,article,journal-article,2000
1,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,article,journal-article,2000
2,10.1890/1051-0761(2000)010[1251:roteka]2.0.co;2,REDISCOVERY OF TRADITIONAL ECOLOGICAL KNOWLEDG...,10/2000,"[{'creatorType': 'author', 'firstName': 'Fikre...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[SE, CA]","[Traditional knowledge, Environmental resource...","[{'id': '15', 'name': 'Life on land'}]","[Conservation, Biodiversity, and Resource Mana...",True,2000,"[[Fikret Berkes, [CA]], [Johan Colding, [SE]],...","[[https://ror.org/02gfys938, CA], [https://ror...","[[University of Manitoba, CA], [Royal Swedish ...",Ecological Applications,1051-0761,article,journal-article,2000
3,10.1016/j.gloenvcha.2013.12.012,Evaluating knowledge exchange in interdiscipli...,2014,"[{'creatorType': 'author', 'firstName': 'Ioan'...",[{'tag': 'ipbes-global_chapter1'}],[GB],"[Knowledge management, Stakeholder, Knowledge ...","[{'id': '15', 'name': 'Life on land'}]","[Sustainability and Climate Change Governance,...",True,2014,"[[Ioan Fazey, [GB]], [Lukas Bunse, [GB]], [Jos...","[[https://ror.org/03h2bxq36, GB], [https://ror...","[[University of Dundee, GB], [University of Du...",Global Environmental Change,0959-3780,article,journal-article,2014
4,10.5751/es-07868-200344,Multiple telecouplings and their complex inter...,2015,"[{'creatorType': 'author', 'firstName': 'Jiang...","[{'tag': 'ipbes-global_chapter1'}, {'tag': 'ip...","[AT, CN, US]","[Zhàng, Ecology, Humanities, Philosophy, Mathe...",None,"[Land Use and Ecosystem Services, Species Dist...",True,2015,"[[Jianguo Liu, [US]], [Vanessa Hull, [US]], [J...","[[https://ror.org/05hs6h993, US], [https://ror...","[[Michigan State University, US], [Michigan St...",Ecology and Society,1708-3087,article,journal-article,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,10.1016/j.tree.2017.06.014,Unifying Research on Social–Ecological Resilie...,2017,"[{'creatorType': 'author', 'firstName': 'Graem...",[{'tag': 'ipbes-global_glossary'}],"[AU, SE]","[Resilience (materials science), Ecological sy...","[{'id': '15', 'name': 'Life on land'}]","[Ecosystem dynamics and resilience, Climate Ch...",True,2017,"[[Graeme S. Cumming, [AU]], [Garry Peterson, [...","[[https://ror.org/028cdc266, AU], [https://ror...",[[ARC Centre of Excellence for Coral Reef Stud...,Trends in Ecology & Evolution,0169-5347,review,journal-article,2017
6359,10.2307/2388682,The Structure and Diversity of Rain Forests at...,1991,"[{'creatorType': 'author', 'firstName': 'Don',...",[{'tag': 'ipbes-global_glossary'}],[None],"[Rainforest, Geography, Tropical rain forest, ...","[{'id': '15', 'name': 'Life on land'}]","[Plant and soil sciences, Environme

In [15]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: [{'chapter': str(x['tag']).replace('ipbes-global_chapter','')} for x in row['tags']], axis=1)

In [16]:
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df_ipbes.loc[:,'rors']=df_ipbes.loc[:,'rors'].apply(lambda x: None if aplatir(x)==[] else x)

In [17]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'year': lambda x: list(x)[0], 'title': lambda x: list(x)[0], 'rors': lambda x: list(x)[0],'authors_name': lambda x: list(x)[0], 'institutions_names': lambda x: list(x)[0],'ipbes': lambda x: list(x)[0], 'countries': lambda x: list(x)[0], 'topics': lambda x: list(x)[0]})
df_ipbes_group=df_ipbes_group.reset_index()

In [18]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [19]:
len(df_ipbes_group.doi.drop_duplicates())==len(df_ipbes_group.doi)

True

In [20]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')

In [21]:
df_ipcc_ipbes['countries']=df_ipcc_ipbes.apply(lambda row: get_xy('countries',row), axis=1)
df_ipcc_ipbes['year']=df_ipcc_ipbes.apply(lambda row: get_xy('year',row), axis=1)
df_ipcc_ipbes['title']=df_ipcc_ipbes.apply(lambda row: get_xy('title',row), axis=1)
df_ipcc_ipbes['rors']=df_ipcc_ipbes.apply(lambda row: get_xy('rors',row), axis=1)
df_ipcc_ipbes['authors_name']=df_ipcc_ipbes.apply(lambda row: get_xy('authors_name',row), axis=1)
df_ipcc_ipbes['institutions_names']=df_ipcc_ipbes.apply(lambda row: get_xy('institutions_names',row), axis=1)
df_ipcc_ipbes['topics']=df_ipcc_ipbes.apply(lambda row: get_xy('topics',row), axis=1)

In [22]:
df_ipcc_ipbes=df_ipcc_ipbes[['doi', 'year','title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes','topics']]

In [23]:
len(df_ipcc_ipbes.doi.drop_duplicates())==len(df_ipcc_ipbes.doi)

True

In [24]:
df_ipcc_ipbes

,doi,year,title,rors,authors_name,institutions_names,ipcc,countries,ipbes,topics
0,10.1001/jama.2012.108940,2012.0,Emergency preparedness and public health: the ...,"[[https://ror.org/05cf8a891, US], [https://ror...","[[Tia Powell, [US]], [Dan Hanfling, [US]], [La...","[[Albert Einstein College of Medicine, US], [I...","[{'name': 'wg2_chap_06', 'wg': 'wg2', 'chap': ...",[US],NaN,"[Disaster Response and Management, Disaster Ma..."
1,10.1001/jama.2014.13186,2014.0,Climate change: challenges and opportunities f...,"[[https://ror.org/03e3qgk42, US], [https://ror...","[[Jonathan A. Patz, [US]], [Howard Frumkin, [U...","[[University of Wisconsin Health, US], [Univer...","[{'name': 'wg2_chap_07', 'wg': 'wg2', 'chap': ...","[GB, US]",NaN,"[Climate Change and Health Impacts, Air Qualit..."
2,10.1001/jama.2014.15715,2014.0,Cause-specific risk of hospital admission rela...,"[[https://ror.org/03vek6s52, US], [https://ror...","[[Jennifer F. Bobb, [US]], [Ziad Obermeyer, [U...","[[Harvard University, US], [Brigham and Women'...","[{'name': 'wg2_chap_07', 'wg': 'wg2', 'chap': ...",[US],NaN,"[Climate Change and Health Impacts, Thermoregu..."
3,10.1001/jama.2020.1097,2020.0,"The Novel Coronavirus Originating in Wuhan, Ch...","[[https://ror.org/05vzafd60, US], [https://ror...","[[Alexandra Phelan, [US]], [Rebecca Katz, [US]...","[[Georgetown University, US], [Center for Glob...","[{'name': 'wg2_chap_07', 'wg': 'wg2', 'chap': ...",[US],NaN,"[COVID-19 epidemiological studies, COVID-19 Pa..."
4,10.1001/jama.2020.15398,2020.0,Cascading Risks of COVID-19 Resurgence During ...,"[[https://ror.org/02dgjyy92, US], [https://ror...","[[James M. Shultz, [US]], [Craig Fugate, [US]]...","[[University of Miami, US], [Craig Technologie...","[{'name': 'wg2_chap_04', 'wg': 'wg2', 'chap': ...",[US],NaN,"[Tropical and Extratropical Cyclones Research,..."
...,...,...,...,...,...,...,...,...,...,...
56994,10.7721/chilyoutenvi.22.2.0144,NaN,Children's use of nature in New Zealand playgr...,NaN,"[[Rebecca J. Sargisson, []], [Ian G. McLean, []]]",[],NaN,NaN,[{'chapter': '2.3_ncp'}],"[Urban Green Space and Health, Children's Righ..."
56995,10.7721/chilyoutenvi.24.2.0010,NaN,The Benefits of Children's Engagement with Nat...,NaN,"[[Tim Gill, []]]",[],NaN,NaN,[{'chapter': '2.3_ncp'}],"[Urban Green Space and Health, Environmental E..."
56996,10.7770/cuhso-v27n1-art1134,NaN,Living the territoriality: Mapuche tourism and...,"[[https://ror.org/04h4z8k05, FR]]","[[Dorian Rommens, [FR]]]","[[UNESCO, FR]]",NaN,[FR],[{'chapter': '3'}],"[Indigenous Cultures and History, Migration, E..."
56997,10.9755/ejfa.v25i1.15376,NaN,Wheat Landraces: A mini review,"[[https://ror.org/02d2m2044, US]]","[[Abdullah A. Jaradat, [US]]]","[[Agricultural Research Service, US]]",NaN,[US],[{'chapter': '3'}],"[Wheat and Barley Genetics and Pathology, Agro..."


In [28]:
update_bso_publications()

2025-05-19 10:08:41,522 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files


2025-05-19 10:11:56,523 | code_utils.BSO | DEBUG | writing 1980029 dois info from bso publications


DEBUG:code_utils.BSO:writing 1980029 dois info from bso publications


In [29]:
bso=pd.DataFrame(get_bso_publications())

In [30]:
df_bso=bso.transpose().reset_index()

In [31]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

In [33]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left').rename(columns={'rors_x':'rors'})

In [35]:
del df_ipcc_ipbes_bso['rors_y']

In [36]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [37]:
df_ipcc_ipbes_bso.columns

Index(['doi', 'year', 'title', 'rors', 'authors_name', 'institutions_names',
       'ipcc', 'countries', 'ipbes', 'topics', 'bso_local_affiliations'],
      dtype='object')

In [25]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso2_iso3.json')

In [ ]:
#df_iso=df_iso.transpose().reset_index().rename(columns={'index':'code',0:'name'})

In [26]:
dict_iso=df_iso.to_dict(orient='records')[0]
dict_iso['XK']='XKX'

In [ ]:
# dict_iso={}
# for i,row in df_iso.iterrows():
#     dict_iso[row['code']]=row['name']
# dict_iso['CW']='Curaçao'
# dict_iso['RS']='Serbia'
# dict_iso['ME']='Montenegro'
# dict_iso['XK']='Kosovo'
# dict_iso['SS']='South Soudan'
# dict_iso['MF']='Saint Martin'

In [38]:
df_ipcc_ipbes_bso.loc[:,'countries']=df_ipcc_ipbes_bso.loc[:,'countries'].apply(lambda x: [dict_iso[y] for y in x] if (x!=None)&(x!=[])&(isinstance(x,float)==False) else None)

In [ ]:
df_brevet_doi=pd.read_json('brevet_doi.json')

In [ ]:
df_ipcc_ipbes_bso_brevet=pd.merge(df_ipcc_ipbes_bso,df_brevet_doi, on='doi', how='left')

In [ ]:
len(df_ipcc_ipbes_bso_brevet.doi.drop_duplicates())==len(df_ipcc_ipbes_bso_brevet.doi)

In [ ]:
df_ipcc_ipbes_bso_brevet.doi.value_counts()[:10]

In [40]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\caches\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

54816 data in cached openalex data
5690 data in cached openalex data


In [41]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [42]:
len(df_ipcc_ipbes_bso.doi.drop_duplicates())==len(df_ipcc_ipbes_bso.doi)

True

In [43]:
current_dict = df_ipcc_ipbes_bso.to_dict(orient='records')

In [44]:
list_bso_ipcc_ipbes=[]
for i in range(len(df_ipcc_ipbes_bso)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [45]:
df_elastic_input=pd.DataFrame(current_dict)

In [46]:
df_elastic_input=df_ipcc_ipbes_bso

In [47]:
df_elastic_input=df_elastic_input[['doi', 'year', 'title', 'rors', 'ipcc','authors_name', 'institutions_names',
       'countries', 'ipbes','topics']]

In [48]:
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[None]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[None]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[None]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[None]),'rors']=None
df_elastic_input.loc[df_elastic_input.authors_name.apply(lambda x: x==[]),'authors_name']=None
df_elastic_input.loc[df_elastic_input.countries.apply(lambda x: x==[]),'countries']=None
df_elastic_input.loc[df_elastic_input.institutions_names.apply(lambda x: x==[]),'institutions_names']=None
df_elastic_input.loc[df_elastic_input.rors.apply(lambda x: x==[]),'rors']=None
df_elastic_input.loc[df_elastic_input.year=='XXXX','year']=None

labs

In [59]:
struct_paysage=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\fr_esr_paysage_structures_all.json')
ror_rnsr_paysage=struct_paysage[['uo_lib','element_ror','identifiant_rnsr']].dropna(subset=['identifiant_rnsr']).rename(columns={'element_ror':'ror'}).dropna(subset=['ror','uo_lib'])

In [72]:
list_ror=ror_rnsr_paysage[['ror','uo_lib']].to_dict(orient='records')

In [ ]:
dict_ror={ l['ror']:l['uo_lib'] for l in list_ror}

In [83]:
x=df_elastic_input.rors[0]
[[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x]

[[None, 'US'], [None, 'US'], [None, 'US'], [None, 'US']]

In [90]:
df_elastic_input['labs']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)

C:\Users\haallat\AppData\Local\Temp\ipykernel_9260\701975774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_elastic_input['labs']=df_elastic_input.loc[:,'rors'].apply(lambda x: [[dict_ror[i[0]],i[1]] if i[0] in list(dict_ror.keys()) else [None,i[1]] for i in x ]if isinstance(x,list) else None)


In [128]:
df_elastic_input['french_labs']=df_elastic_input.loc[:,'labs'].apply(lambda x: list(pd.Series([i[0] for i in x if i[1]=='FR' and pd.isna(i[0])==False]).drop_duplicates()) if isinstance(x,list) else None)

In [133]:
len(aplatir(list(df_elastic_input['french_labs'].dropna())))

2975

institutions

In [ ]:
ror_rnsr_paysage=struct_paysage[['element_ror','identifiant_rnsr']].dropna(subset=['identifiant_rnsr']).rename(columns={'element_ror':'ror'})
ror_rnsr_paysage.loc[:,'ror']=ror_rnsr_paysage.loc[:,'ror'].apply(lambda x: str(x).replace('https://ror.org/','') if pd.isna(x)==False else None)

In [146]:
rnsr_tutelles_scanr=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\organizations.jsonl', lines=True)
rnsr_tutelles_scanr['tutelles_label']=rnsr_tutelles_scanr['institutions'].apply(lambda x: [y.get('label') for y in x] if len(x)>0 else None)
rnsr_tutelles_scanr['tutelles_id']=rnsr_tutelles_scanr['institutions'].apply(lambda x: [y.get('structure') for y in x] if len(x)>0 else None)
rnsr_tutelles_scanr.loc[:,'ror']=rnsr_tutelles_scanr.loc[:,'externalIds'].apply(lambda y: [x.get('id') for x in y if x.get('type')=='ROR'])
rnsr_tutelles_scanr.loc[:,'rnsr']=rnsr_tutelles_scanr.loc[:,'externalIds'].apply(lambda y: [x.get('id') for x in y if x.get('type')=='rnsr'])
rnsr_tutelles=rnsr_tutelles_scanr[['id','tutelles_label']].rename(columns={'id':'rnsr'})
ror_tutelles=rnsr_tutelles_scanr[['ror','tutelles_label']].explode('ror')

In [147]:
rnsr_tutelles_scanr

,id,startDate,creationYear,endDate,status,label,acronym,kind,level,legalCategory,...,links,externalIds,keywords,activities,createdAt,removedAt,tutelles_label,tutelles_id,ror,rnsr
0,200012318,2008-01-01T00:00:00,2008.0,2018-12-31T00:00:00,old,"{'fr': 'ETABLISSEMENT DE L'ELEVAGE 04 06 83', ...","{'fr': 'OIER', 'default': 'OIER'}",[Secteur public],Établissement public administratif,"{'code': '7379', 'label': None}",...,[],"[{'id': '200012318', 'type': 'sirene'}, {'id':...",{},[],2019-01-21T10:18:40,2019-01-21T10:18:40,None,None,[],[]
1,200034528,2013-01-01T00:00:00,2013.0,None,active,{'fr': 'Centre Hospitalier Universitaire de Ma...,"{'fr': 'CHU Martinique', 'default': 'CHU Marti...",[Secteur public],Établissement public de santé,"{'code': '7364', 'label': None}",...,[{'url': 'https://fr.wikipedia.org/wiki/Centre...,"[{'id': '200034528', 'type': 'sirene'}, {'id':...",{},[],2019-01-21T10:21:11,None,None,None,[],[]
2,200035335,2012-06-27T00:00:00,2012.0,None,active,{'fr': 'Pôle métropolitain Nantes Saint-Nazair...,{},[Secteur public],Collectivité territoriale ou assimilé,"{'code': '7340', 'label': None}",...,"[{'type': 'main', 'url': 'https://www.agence-a...","[{'id': '200035335', 'type': 'sirene'}, {'id':...",{},[],2019-01-21T10:21:23,None,None,None,[],[]
3,200046977,2015-01-01T00:00:00,2015.0,None,active,"{'fr': 'Métropole de Lyon', 'default': 'Métrop...",{},[Secteur public],Collectivité territoriale ou assimilé,"{'code': '7229', 'label': None}",...,[{'url': 'https://www.grandlyon.com/pratique/f...,"[{'id': '200046977', 'type': 'sirene'}, {'id':...",{},[],2019-01-21T10:22:12,None,None,None,[],[]
4,226400018,1982-01-01T00:00:00,1982.0,None,active,"{'fr': 'Département des Pyrénées Atlantiques',...",{},[Secteur public],Collectivité territoriale ou assimilé,"{'code': '7220', 'label': None}",...,"[{'url': 'http://www.le64.fr/accueil/rss.xml',...","[{'id': '226400018', 'type': 'sirene'}, {'id':...",{},[],2019-01-21T10:27:22,None,None,None,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73009,202424495A,2024-01-01T00:00:00,2024.0,None,active,{'fr': 'Fédération Occitane de Recherche Mathé...,"{'fr': 'OcciMath', 'default': 'OcciMath'}",[Structure de recherche],Unité de recherche,{},...,[],"[{'id': '202424495A', 'type': 'rnsr'}, {'id': ...",{},[],2024-03-01T20:36:42,None,[Institut National des Sciences Appliquées Tou...,"[193101524, 193113842, 196604375, 180089013, 1...",[],[202424495A]
73010,202424497C,2024-01-01T00:00:00,2024.0,None,active,"{'fr': 'Littoral Urbain Durable Intelligent', ...","{'fr': 'LUDI', 'default': 'LUDI'}",[Structure de recherche],Unité de recherche,{},...,[],"[{'id': '202424497C', 'type': 'rnsr'}, {'id': ...",{},[],2024-03-01T20:36:44,None,"[Université La Rochelle, Centre national de la...","[191700327, 180089013]",[],[202424497C]
73011,202424503J,2024-01-01T00:00:00,2024.0,None,active,{'fr': 'Membrane Signalling and Inflammation i...,"{'fr': 'ISCHEMIA', 'default': 'ISCHEMIA'}",[Structure de recherche],Unité de recherche,{},...,[],"[{'id': '202424503J', 'type': 'rnsr'}, {'id': ...",{},"[{'code': 'LS7', 'type': 'panel_erc', 'label':...",2024-03-01T20:36:45,None,"[Université de Tours, Institut national de la ...","[193708005, 180036048]",[],[202424503J]
73012,202424515X,2024-01-01T00:00:00,2024.0,None,active,{'fr': 'Centre de Recherche sur les Expertises...,"{'fr': 'CREAT', 'default': 'CREAT'}",[Structure de recherche],Unité de recherche,{},...,[],"[{'id': '202424515X', 'type': 'rnsr'}]",{},"[{'code': 'SH3', 'type': 'panel_erc', 'label':...",2024-03-01T20:36:46,None,[Université de Lorraine],[130015506],[],[202424515X]


In [136]:
len(df_elastic_input)

56994

In [143]:
aplatir(list(df_elastic_input[pd.isna(df_elastic_input.countries)].french_labs.dropna()))

[]

In [144]:
df_elastic_input=df_elastic_input.dropna(subset=['countries'])

In [ ]:
df_elastic_input[df_elastic_input.countries.apply(lambda x: 'FRA' in x)].reset_index()#.doi.to_json('list_doi_fr.json',orient='records')

In [ ]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

fichier excel

In [ ]:
df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
df=df[['doi','ipcc','ipbes']]

In [ ]:
df.loc[:,'ipbes']=df.loc[:,'ipbes'].apply(lambda x: [y['chapter'] for y in x] if isinstance(x, list) else [])

In [ ]:
df.loc[:,'ipcc']=df.loc[:,'ipcc'].apply(lambda x: [y['name'] for y in x] if isinstance(x, list) else [])

In [ ]:
df.loc[:,'doi']=df.loc[:,'doi'].apply(lambda x: f"doi{x}")

In [ ]:
df.drop_duplicates('doi')

In [ ]:
with open('ipcc_ipbes_chapters.jsonl', 'w') as f:
    for record in df.to_dict(orient='records'):
        record2 = {k: v for k, v in record.items() if isinstance(v, list) or isinstance(v, str)}
        json.dump(record2, f)
        f.write('\n')

In [ ]:
df.loc[:,'rors']=df.loc[:,'rors'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'authors_name']=df.loc[:,'authors_name'].apply(lambda x: [[]] if x==[None] else x)
df.loc[:,'institutions_names']=df.loc[:,'institutions_names'].apply(lambda x: [[]] if x==[None] else x)

In [ ]:
df.loc[pd.isna(df['rors'])==False,'rors']=df.loc[pd.isna(df['rors'])==False,'rors'].apply(lambda x: ','.join([j.replace('https://ror.org/','') for j in list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())]))

In [ ]:
df.loc[pd.isna(df['institutions_names'])==False,'institutions_names']=df.loc[pd.isna(df['institutions_names'])==False,'institutions_names'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y[i].keys()) for y in x for i in range(len(y))])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['authors_name'])==False,'authors_name']=df.loc[pd.isna(df['authors_name'])==False,'authors_name'].apply(lambda x: ','.join(list(pd.Series(aplatir([list(y.keys()) for y in x if isinstance(y,dict)])).drop_duplicates())))

In [ ]:
df.loc[pd.isna(df['ipcc'])==False,'ipcc']=df.loc[pd.isna(df['ipcc'])==False,'ipcc'].apply(lambda x: ','.join(list((pd.Series([y.get('name') for y in x]).drop_duplicates()))))

In [ ]:
df.loc[pd.isna(df['ipbes'])==False,'ipbes']=df.loc[pd.isna(df['ipbes'])==False,'ipbes'].apply(lambda x: ','.join(list((pd.Series([y.get('chapter') for y in x]).drop_duplicates()))))

In [ ]:
df['is_fr']=df['countries'].apply(lambda x: 'France' in x if isinstance(x,list) else False)

In [ ]:
df.loc[pd.isna(df['countries'])==False,'countries']=df.loc[pd.isna(df['countries'])==False,'countries'].apply(lambda x: ','.join(x))

In [ ]:
df[['doi','title','ipcc','ipbes','countries','is_fr','rors','institutions_names','authors_name']].to_excel('doi_ipcc_ipbes.xlsx', index=False)